In [ ]:
from src.pyAOM_utils import *

In [ ]:
# STO parameters for GTO-STO projection step
STO_proj_dict={
    'H':{'STOs':1  ,'1s':1.0000},
    'C':{'STOs':1+3,'2s':1.6083,'2p':1.442657},
    'N':{'STOs':1+3,'2s':1.9237,'2p':1.646703},
    'O':{'STOs':1+3,'2s':2.2458,'2p':1.858823},
    'F':{'STOs':1+3,'2s':2.5638,'2p':2.136394},
    'S':{'STOs':1+3,'3s':2.1223,'3p':1.651749},
}
# general configuration
config={
    'label': 'TTF',
    'single_mol_xyz': 'single_molecules/TTF.xyz',
    'basis': 'DZVP-GTH',
    'potential': 'GTH-PBE',
    'MO': 26,
    'cp2k_template': 'templates/sp_GGA_template.inp',
    'cp2k_basis_file': 'cp2k_files/GTH_BASIS_SETS',
}
# CP2K DFT parameters
CP2K_QS_GGA_parameters={
    'PROJECT_NAME': 'TTF_PBE_DZVP-GTH',
    'XC_FUNCTIONAL': 'PBE',
    'BASIS_SET_FILE_NAME': 'GTH_BASIS_SETS',
    'POTENTIAL_FILE_NAME': 'POTENTIAL',
    'MGRID_CUTOFF': 450.0,
    'MGRID_REL_CUTOFF': 75.0,
    'SCF_MAX_SCF': 200,
    'MO_CUBES_NHOMO': 6,
    'MO_CUBES_NLUMO': 6,
    'MO_CUBES_STRIDE': 5,
}
# default cp2k:qs output path
cp2k_out=f'cp2k_output/{CP2K_QS_GGA_parameters["PROJECT_NAME"]}.out'

In [ ]:
# read a single molecule xyz file
mymol=single_molecule(config['single_mol_xyz'])
# prepare CP2K:QS input file for single molecule
mymol.prep_cp2k_single(CP2K_QS_GGA_parameters,config['label'],config['cp2k_template'],config['basis'],config['potential'])
print(f'Note: make sure CP2K:QS output gets in "{cp2k_out}"')

In [ ]:
# provided we carried out the DFT calculation and placed the CP2K output in the appropriate directory,
# read CP2K:QS output and retrieve MO info
mymol.get_cp2k_info(config['MO'],cp2k_out,config['cp2k_basis_file'],config['basis'])

In [ ]:
# STO init and overlap matrix calculation
mymol.initialize_STOs(STO_proj_dict)
# carry out GTO->STO projection
mymol.project()

In [ ]:
# store AOM coefficients
mymol.save_AOM(config['label'])

In [ ]:
# projection completeness
mymol.orb_compl_dict

In [ ]:
# cube file preview; optional
mymol.cube_preview(STO_proj_dict,f'output/{config["label"]}/{config["label"]}_{config["MO"]}.cube')

In [ ]:
# define STO parameters for AOM calculations
AOM_dict={
    'H':{'STOs':1  ,'1s':1.0000},
    'C':{'STOs':1+3,'2s':1.6083,'2p':1.385600},
    'N':{'STOs':1+3,'2s':1.9237,'2p':1.617102},
    'O':{'STOs':1+3,'2s':2.2458,'2p':1.505135},
    'F':{'STOs':1+3,'2s':2.5638,'2p':1.665190},
    'S':{'STOs':1+3,'3s':2.1223,'3p':1.641119},
}

In [ ]:
# locate dimers: define the directory with dimers
# suggested file structure: create a directory with the molecule name in 'dimers'
# and use the following naming for each dimer: <name>_<No>_<comment>.xyz
dimer_dir='dimers/TTF'
# define the AOM files for both dimer fragments
frag1_AOM_file='output/TTF/AOM_COEFF.dat'
frag2_AOM_file='output/TTF/AOM_COEFF.dat'
# define MOs
frag1_MO=26
frag2_MO=26
#--------------------------------------------------------------------------------
Sab_array=[]
for file in sorted([i for i in os.listdir(dimer_dir) if i.endswith('.xyz')==True]):
    dimer_xyz_file=f'{dimer_dir}/{file}'
    val=Sab(dimer_xyz_file,frag1_AOM_file,frag2_AOM_file,frag1_MO,frag2_MO,AOM_dict)
    Sab_array.append(val)
    print(f'{file} Sab: {val}')

In [ ]:
# TTF-specific test
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set_context("talk")
# this is the generic C scaling constant for AOM
C=9779.738447557258
# this is an empirical scaling for PBE/DZVP-GTH CP2K POD HAB as to correlate with ab-initio reference data
HAB_scale=1.325
# TTF PBE/DZVP-GTH POD couplings
CP2K_HAB=[410.72,
 200.907,
 99.4675,
 50.0264,
 146.124,
 60.6216,
 80.0497,
 0.0032636,
 4.14261e-05,
 95.4641,
 3.01126]
# plot!
plt.plot([abs(i)*C for i in Sab_array],[i*HAB_scale for i in CP2K_HAB],'*')
plt.plot([0,600],[0,600])
plt.xlabel('Predicted AOM HAB (meV)')
plt.ylabel('CP2K POD HAB (scaled by 1.325) (meV)');